# SVI: Conditinal Independence, Subsampling, Amortization

这一部分主要讨论如何提高SVI的效率。

目标是希望将SVI应用到大型的Datasets上。

比如，类似DNN中的minibatch-backpropagation。只要在给定prior的条件下，X的分布在不同的样本间是独立的，则就可以使用下面的操作：


$$\sum_{i=1}^N{\log{p(X_i|Z)}}\approx\frac{N}{M}\sum_{i\in{\mathscr{i}_M}}{\log{p(X_i|Z)}}$$

其中$\mathscr{i}_M$是样本量大小为M的mini-batch。即每次进行参数更新的时候，只是使用其中的随机一部分样本。

## Marking Conditional Independence in Pyro

首先第一步，我们需要在model和guide中指定条件独立性：

In [3]:
# 这是我们之前的做法
import pyro
import pyro.distributions as dist

def model(data):
    f = pyro.sample("latent_fairness", dist.Beta(10.0, 10.0))
    for i in range(len(data)):
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

其实精确定义条件独立性的方法，就是使用`pyor.plate`来替代`range`，注意，此时需要给与一个唯一的名称：

In [4]:
def model(data):
    f = pyro.sample("latent_fairness", dist.Beta(10, 10))
    for i in pyro.plate("data_loop", len(data)):
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

注意，使用的时候不用再外面再嵌套`list`等东西，比如：`list(pyro.plate("data_loop", len(data)))`。

除了上面的seq版`plate`，还有一个速度更快的矢量版`plate`：

In [6]:
import torch

data = torch.zeros(10)
data[0:6] = torch.ones(6)  # 首先需要观察到的obs_data是tensor

def model(data):
    f = pyro.sample("latent_fairness", dist.Beta(10.0, 10.0))
    with pyro.plate("observe_data"):  # 直接把plate当做一个上下文管理器
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

## Subsampling

这里有多种方式实现子采样minibatch

第一种依然是依赖于`plate`，在seq版本中，只需要给一个参数`subsample_size`即可。

In [8]:
def model(data):
    f = pyro.sample("latent_fairness", dist.Beta(10, 10))
    for i in pyro.plate("data_loop", len(data), subsample_size=5):
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

model(data)

另一个矢量化版本中，需要补充N、minibatch的大小，然后使用其提供的一个ind来实现：

In [11]:
def model(data):
    f = pyro.sample("latent_fairness", dist.Beta(10.0, 10.0))
    with pyro.plate("observe_data", size=10, subsample_size=5, device="cuda:0") as ind:  # 直接把plate当做一个上下文管理器
        pyro.sample("obs", dist.Bernoulli(f), obs=data[ind])
    print(ind)

model(data)

tensor([7, 2, 6, 3, 8], device='cuda:0')


注意到，`plate`可以提供一个`device`参数来将提供的ind放到GPU上，如果data是在gpu上的话。

另外，其还有一个`subsample`参数来细致的控制子采样过程。

以上进行的条件独立性只发生在model中，因为条件独立只是X的条件独立。那如果Z中有一部分也遵循条件独立的条件，则在model和guide中都要进行条件独立的设置。

假设现在我们的模型是这样的：

$$
p(X, Z, \beta) = p(\beta)\prod_{i=1}^N{p(X_i|Z_i)p(Z_i|\beta)}
$$

因为而我们许可估计的prior或latent variables对于每个人都有一个独立的变量。所以，我们的guide有下面的形式：

$$
q(Z, \beta) = q(\beta)\prod_{i=1}^N{q(Z_i|\beta, \lambda_i)}
$$

其中的lambda就是每个分布中可以进行梯度下降调整的可训练参数。

In [12]:
def model(data):
    beta = pyro.sample("beta", ...) # sample the global RV
    for i in pyro.plate("locals", len(data)):
        z_i = pyro.sample("z_{}".format(i), ...)
        # compute the parameter used to define the observation
        # likelihood using the local random variable
        theta_i = compute_something(z_i)
        pyro.sample("obs_{}".format(i), dist.MyDist(theta_i), obs=data[i])

In [ ]:
def guide(data):
    beta = pyro.sample("beta", ...) # sample the global RV
    for i in pyro.plate("locals", len(data), subsample_size=5):
        # sample the local RVs
        pyro.sample("z_{}".format(i), ..., lambda_i)

因为在guide和model中出现了相同的"locals"，而其就是需要进行subsampling的对象，则只需要指定一个即可。

## Amortization

如果我们的样本量非常大，则每个样本都分配一个可训练变分参数就会非常使得参数空间急剧增大。而更加有效的方法是通过将样本数据进行映射来得到这些参数（就像VAE那样。）

$$q(\beta)\prod_{n-1}^N{q(Z_i|f(X_i))}$$